In [230]:
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
from pymongo import MongoClient
import json
import requests
import os

In [231]:
token = os.getenv("token")

In [232]:
client = MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("usa_companiess")
c.find_one()

{'_id': ObjectId('654624e51d17ce5e07c16dd7'),
 'name': 'hi5',
 'category_code': 'games_video',
 'number_of_employees': 100,
 'offices': [{'city': 'San Francisco',
   'country_code': 'USA',
   'latitude': 37.788668,
   'longitude': -122.400558}]}

In [233]:
path = "data/usa_companiess.json"
df = pd.read_json(path)
df

,name,category_code,number_of_employees,offices
0,hi5,games_video,100,"[{'city': 'San Francisco', 'country_code': 'US..."
1,Social Gaming Network,games_video,100,"[{'city': 'Los Angeles', 'country_code': 'USA'..."
2,VBS TV,games_video,100,"[{'city': 'Brooklyn', 'country_code': 'USA', '..."
3,ZillionTV,games_video,100,"[{'city': 'Sunnyvale', 'country_code': 'USA', ..."
4,Exent,games_video,100,"[{'city': 'New York', 'country_code': 'USA', '..."
5,iWin,games_video,103,"[{'city': 'San Francisco', 'country_code': 'US..."
6,RockYou,games_video,106,"[{'city': 'Redwood City', 'country_code': 'USA..."
7,G5 Entertainment,games_video,107,"[{'city': 'Kharkov', 'country_code': 'UKR', 'l..."
8,Zynga,games_video,115,"[{'city': 'San Francisco', 'country_code': 'US..."
9,Livestream,games_video,120,"[{'city': 'New York', 'country_code': 'USA', '..."


### 1st city: New York, NY

In [234]:
#Polygons??
# location = [40.761979, -74.026033]
# position = {"type": "Point", "coordinates":location}
# projection = {"name": 1, "_id": 0}
# c.find_one({"geometry":{"$geoIntersects":{"$geometry":position}}}, projection)["name"]
# Tried to do GeoJSON but I had problems with indexes in Mongo

In [235]:
lat = 40.726155
lon = -73.995625
radius = 3500

def requests_for_foursquare(query, lat, lon,radius):
        url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}"

        headers = {"accept": "application/json","Authorization": token}
        try:
            return requests.get(url, headers=headers).json()
        except Exception as e:
            print("Error:", e)
            return None
        
def requests_for_foursquare_2queries(query1,query2, lat, lon,radius):
        url = f"https://api.foursquare.com/v3/places/search?query={query1}%20%26%20{query2}&ll={lat}%2C{lon}&radius={radius}"

        headers = {"accept": "application/json","Authorization": token}
        try:
            return requests.get(url, headers=headers).json()
        except Exception as e:
            print("Error:", e)
            return None
query = ["starbucks","schools","airport","club","vegan","basketball"]
query1= ["pet"]
query2 = ["grooming"]

In [236]:
lat = 40.726155
lon = -73.995625
radius = 3500

def apply_request_starbucks():
    ny_starbucks = []
    for i in query: 
        if i == "starbucks":
            result = requests_for_foursquare("starbucks",lat,lon,radius)
            ny_starbucks.append(result)
    return ny_starbucks

def apply_request_school():
    ny_schools = []
    for i in query:
        if i == "schools":
            result = requests_for_foursquare("schools",lat,lon,radius)
            ny_schools.append(result)
    return ny_schools

def apply_request_airports():
    ny_airport = []
    for i in query:
        if i == "airport":
            result = requests_for_foursquare("airport",lat,lon,radius)
            ny_airport.append(result)
    return ny_airport

def apply_request_clubs():
    ny_clubs = []
    for i in query:
        if i == "club":
            result = requests_for_foursquare("club",lat,lon,radius)
            ny_clubs.append(result)
    return ny_clubs

def apply_request_vegan():
    ny_vegan = []
    for i in query:
        if i == "vegan":
            result = requests_for_foursquare("vegan",lat,lon,radius)
            ny_vegan.append(result)
    return ny_vegan

def apply_request_basketball():
    ny_basketball = []
    for i in query:
        if i == "basketball":
            result = requests_for_foursquare("basketball",lat,lon,radius)
            ny_basketball.append(result)
    return ny_basketball

def apply_request_pet():
    ny_pet = []
    result = requests_for_foursquare_2queries("pet","grooming",lat,lon,radius)
    ny_pet.append(result)  
    return ny_pet

In [237]:
new_dict = {}

ny_starbucks = apply_request_starbucks()
starbucks_names = [result["name"] for i in ny_starbucks for result in i.get("results", [])]
new_dict["Starbucks_n"] = starbucks_names
starbucks_latitudes = [result["geocodes"]["main"]["latitude"] for i in ny_starbucks for result in i.get("results", [])]
new_dict["Starbucks_lat"] = starbucks_latitudes
starbucks_longitudes = [result["geocodes"]["main"]["longitude"] for i in ny_starbucks for result in i.get("results", [])]
new_dict["Starbucks_lon"] = starbucks_longitudes

ny_schools = apply_request_school()
schools_names = [result["name"] for i in ny_schools for result in i.get("results", [])]
new_dict["Schools_n"] = schools_names
schools_latitudes = [result["geocodes"]["main"]["latitude"] for i in ny_schools for result in i.get("results", [])]
new_dict["Schools_lat"] = schools_latitudes
schools_longitudes = [result["geocodes"]["main"]["longitude"] for i in ny_schools for result in i.get("results", [])]
new_dict["Schools_lon"] = schools_longitudes

ny_airport = apply_request_airports()
airport_name = [result["name"] for i in ny_airport for result in i.get("results", [])]
new_dict["Airport_n"] = airport_name
airport_latitude = [result["geocodes"]["main"]["latitude"] for i in ny_airport for result in i.get("results", [])]
new_dict["Airport_lat"] = airport_latitude
airport_longitude = [result["geocodes"]["main"]["longitude"] for i in ny_airport for result in i.get("results", [])]
new_dict["Airport_lon"] = airport_longitude

ny_clubs = apply_request_clubs()
clubs_name = [result["name"] for i in ny_clubs for result in i.get("results", [])]
new_dict["Clubs_n"] = clubs_name
club_latitude = [result["geocodes"]["main"]["latitude"] for i in ny_clubs for result in i.get("results", [])]
new_dict["Clubs_lat"] = club_latitude
club_longitude = [result["geocodes"]["main"]["longitude"] for i in ny_clubs for result in i.get("results",[])]
new_dict["Clubs_lon"] = club_longitude

ny_basketball = apply_request_basketball()
basketball_name = [result["name"] for i in ny_basketball for result in i.get("results",[])]
new_dict["Basketball_n"] = basketball_name
basketball_latitude = [result["geocodes"]["main"]["latitude"] for i in ny_basketball for result in i.get("results", [])]
new_dict["Basketball_lat"] = basketball_latitude
basketball_longitde = [result["geocodes"]["main"]["longitude"] for i in ny_basketball for result in i.get("results", [])]
new_dict["Basketball_lon"] = basketball_longitde

ny_pet = apply_request_pet()
pet_name = [result["name"] for i in ny_pet for result in i.get("results",[])]
new_dict["Pet_n"] = pet_name
pet_latitude = [result["geocodes"]["main"]["latitude"] for i in ny_pet for result in i.get("results", [])]
new_dict["Pet_lat"] = pet_latitude
pet_longitde = [result["geocodes"]["main"]["longitude"] for i in ny_pet for result in i.get("results", [])]
new_dict["Pet_lon"] = pet_longitde


In [238]:
ny_companies = pd.DataFrame(new_dict)
ny_companies

,Starbucks_n,Starbucks_lat,Starbucks_lon,Schools_n,Schools_lat,Schools_lon,Airport_n,Airport_lat,Airport_lon,Clubs_n,Clubs_lat,Clubs_lon,Basketball_n,Basketball_lat,Basketball_lon,Pet_n,Pet_lat,Pet_lon
0,Starbucks,40.726874,-74.000192,Third Street Music School Settlement,40.730902,-73.986904,Airport,40.752971,-73.986765,Smalls Jazz Club,40.734411,-74.002717,Basketball Court,40.732496,-73.979550,Pet Central,40.738710,-73.995376
1,Starbucks,40.727270,-73.995536,Trapeze School New York,40.729214,-74.011330,Skyhome,40.717953,-73.985043,The Lower East Side Girls Club,40.723469,-73.976528,Basketball City,40.709975,-73.984905,The Pet Bar,40.726421,-74.000960
2,Starbucks,40.729279,-74.004959,Avenues: The World School,40.749393,-74.003309,Atlantic Aviation,40.743692,-73.972731,New York Comedy Club,40.738889,-73.980798,Hudson River Park Basketball Court,40.718816,-74.012838,Loulou Petit Bistro & Speakeasy,40.742681,-74.000161
3,Starbucks,40.741623,-74.005316,Stuyvesant High School,40.717978,-74.013896,Zinc Bar,40.729656,-73.999036,Gotham Comedy Club,40.744131,-73.996419,Basketball Courts,40.725597,-74.011405,Barba Men's Grooming Boutique,40.742340,-73.998794
4,Starbucks,40.730073,-73.991630,The DiMenna Center for Classical Music,40.756032,-73.997281,King And Queen Limo,40.705624,-74.006117,Salmagundi Club,40.734258,-73.994458,Tompkins Square Park Basketball Courts,40.726974,-73.980550,Pet Central,40.744922,-73.997105
5,Starbucks,40.733063,-74.002707,Grace Church School,40.731880,-73.990610,Balthazar,40.722675,-73.998195,Bowery Ballroom,40.720425,-73.993346,Playground 9- Basketball Court/Sprinklers,40.732175,-73.979706,NYC Pet 171838747,40.718467,-73.963981
6,Starbucks,40.722492,-73.997896,Ps 234 Independence School,40.716155,-74.011510,Alitalia Airlines,40.743114,-73.986160,Ludlow House,40.720505,-73.988417,Masaryk Basketball,40.717319,-73.980566,Walter's Pet Styles,40.745053,-73.978287
7,Starbucks,40.714207,-74.015710,HB Studio,40.736060,-74.007541,Brooklyn Radio Dispatcher,40.714913,-73.959741,Blue Note,40.730962,-74.000615,West 4th Street Courts,40.731534,-74.001380,PS9 Pet Supplies,40.718787,-73.956049
8,Starbucks,40.718154,-74.007443,Battery Park City School,40.706445,-74.017539,Colgate-Jersey City Heliport,40.713684,-74.032842,Mezzrow,40.734650,-74.001853,Madison Square Garden,40.750561,-73.993231,Towne House Grooming,40.746203,-73.997593
9,Starbucks,40.740445,-74.005008,Grace Church High School,40.728679,-73.991313,Shake Shack,40.741548,-73.988195,Village Vanguard,40.736003,-74.001688,British Basketball Association US Headquarters,40.710421,-74.010191,Pet Central Second Avenue,40.747262,-73.974399


In [239]:
lat = 40.726155
lon = -73.995625
ny_map = folium.Map(location = [lat, lon], zoom_start = 15)

In [240]:
def add_marker(map, lat, lon, icon_name, color, icon_color):
    icon = Icon(
        icon=icon_name,
        prefix="fa",
        color=color,
        icon_color=icon_color
    )
    new_marker = folium.Marker(location=[lat, lon], icon=icon)
    new_marker.add_to(map)

add_marker(ny_map,lat,lon,"hand-paper","red","white")

#Ideal office
lat1 = 40.7527262
lon1 = -73.9772294
add_marker(ny_map,lat1,lon1,"train","black","white")

for index, row in ny_companies.iterrows():
    if row["Starbucks_n"] == "Starbucks":
        add_marker(ny_map, row["Starbucks_lat"], row["Starbucks_lon"], "coffee", "white", "green")

    if row["Schools_n"]:
        add_marker(ny_map, row["Schools_lat"], row["Schools_lon"], "graduation-cap", "blue", "white")

    if row["Airport_n"]:
        add_marker(ny_map, row["Airport_lat"], row["Airport_lon"], "plane", "gray", "white")

    if row["Clubs_n"]:
        add_marker(ny_map, row["Clubs_lat"], row["Clubs_lon"], "beer", "lightred", "white")

    if row["Basketball_n"]:
        add_marker(ny_map, row["Basketball_lat"], row["Basketball_lon"], "fa-basketball-ball'", "orange", "black")

    if row["Pet_n"]:
        add_marker(ny_map, row["Pet_lat"], row["Pet_lon"], "paw", "purple", "white")

lat = 40.726155
lon = -73.995625
folium.Circle([lat, lon],
                        radius=3500,
                        fill_color='blue'
                    ).add_to(ny_map)
ny_map

In [241]:
ny_map.save("maps/ny_map.html")

### 2nd city: San Francisco, CA

In [242]:
lat = 37.788668
lon = -122.400558
radius = 2000

def apply_request_starbucks():
    sf_starbucks = []
    for i in query: 
        if i == "starbucks":
            result = requests_for_foursquare("starbucks",lat,lon,radius)
            sf_starbucks.append(result)
    return sf_starbucks

def apply_request_school():
    sf_schools = []
    for i in query:
        if i == "schools":
            result = requests_for_foursquare("schools",lat,lon,radius)
            sf_schools.append(result)
    return sf_schools

def apply_request_airports():
    sf_airport = []
    for i in query:
        if i == "airport":
            result = requests_for_foursquare("airport",lat,lon,radius)
            sf_airport.append(result)
    return sf_airport

def apply_request_clubs():
    sf_clubs = []
    for i in query:
        if i == "club":
            result = requests_for_foursquare("club",lat,lon,radius)
            sf_clubs.append(result)
    return sf_clubs

def apply_request_vegan():
    sf_vegan = []
    for i in query:
        if i == "vegan":
            result = requests_for_foursquare("vegan",lat,lon,radius)
            sf_vegan.append(result)
    return sf_vegan

def apply_request_basketball():
    sf_basketball = []
    for i in query:
        if i == "basketball":
            result = requests_for_foursquare("basketball",lat,lon,radius)
            sf_basketball.append(result)
    return sf_basketball

def apply_request_pet():
    sf_pet = []
    result = requests_for_foursquare_2queries("pet","grooming",lat,lon,radius)
    sf_pet.append(result)  
    return sf_pet


In [243]:
sf_dict = {}

sf_starbucks = apply_request_starbucks()
starbucks_names = [result["name"] for i in sf_starbucks for result in i.get("results", [])]
sf_dict["Starbucks_n"] = starbucks_names
starbucks_latitudes = [result["geocodes"]["main"]["latitude"] for i in sf_starbucks for result in i.get("results", [])]
sf_dict["Starbucks_lat"] = starbucks_latitudes
starbucks_longitudes = [result["geocodes"]["main"]["longitude"] for i in sf_starbucks for result in i.get("results", [])]
sf_dict["Starbucks_lon"] = starbucks_longitudes

sf_schools = apply_request_school()
schools_names = [result["name"] for i in sf_schools for result in i.get("results", [])]
sf_dict["Schools_n"] = schools_names
schools_latitudes = [result["geocodes"]["main"]["latitude"] for i in sf_schools for result in i.get("results", [])]
sf_dict["Schools_lat"] = schools_latitudes
schools_longitudes = [result["geocodes"]["main"]["longitude"] for i in sf_schools for result in i.get("results", [])]
sf_dict["Schools_lon"] = schools_longitudes

sf_airport = apply_request_airports()
airport_name = [result["name"] for i in sf_airport for result in i.get("results", [])]
sf_dict["Airport_n"] = airport_name
airport_latitude = [result["geocodes"]["main"]["latitude"] for i in sf_airport for result in i.get("results", [])]
sf_dict["Airport_lat"] = airport_latitude
airport_longitude = [result["geocodes"]["main"]["longitude"] for i in sf_airport for result in i.get("results", [])]
sf_dict["Airport_lon"] = airport_longitude

sf_clubs = apply_request_clubs()
clubs_name = [result["name"] for i in sf_clubs for result in i.get("results", [])]
sf_dict["Clubs_n"] = clubs_name
club_latitude = [result["geocodes"]["main"]["latitude"] for i in sf_clubs for result in i.get("results", [])]
sf_dict["Clubs_lat"] = club_latitude
club_longitude = [result["geocodes"]["main"]["longitude"] for i in sf_clubs for result in i.get("results",[])]
sf_dict["Clubs_lon"] = club_longitude

sf_basketball = apply_request_basketball()
basketball_name = [result["name"] for i in sf_basketball for result in i.get("results",[])]
sf_dict["Basketball_n"] = basketball_name
basketball_latitude = [result["geocodes"]["main"]["latitude"] for i in sf_basketball for result in i.get("results", [])]
sf_dict["Basketball_lat"] = basketball_latitude
basketball_longitde = [result["geocodes"]["main"]["longitude"] for i in sf_basketball for result in i.get("results", [])]
sf_dict["Basketball_lon"] = basketball_longitde

sf_pet = apply_request_pet()
pet_name = [result["name"] for i in sf_pet for result in i.get("results",[])]
sf_dict["Pet_n"] = pet_name
pet_latitude = [result["geocodes"]["main"]["latitude"] for i in sf_pet for result in i.get("results", [])]
sf_dict["Pet_lat"] = pet_latitude
pet_longitde = [result["geocodes"]["main"]["longitude"] for i in sf_pet for result in i.get("results", [])]
sf_dict["Pet_lon"] = pet_longitde

sf_companies = pd.DataFrame(sf_dict)
sf_companies

,Starbucks_n,Starbucks_lat,Starbucks_lon,Schools_n,Schools_lat,Schools_lon,Airport_n,Airport_lat,Airport_lon,Clubs_n,Clubs_lat,Clubs_lon,Basketball_n,Basketball_lat,Basketball_lon,Pet_n,Pet_lat,Pet_lon
0,Starbucks,37.788398,-122.393628,ELS San Francisco Downtown,37.789242,-122.398840,UCS Limousine and Car Service,37.788091,-122.405368,The Battery,37.797946,-122.401217,Berry Basketball Courts,37.772080,-122.397686,Bernie's Grooming City Dogs,37.783786,-122.389414
1,Starbucks,37.793812,-122.396743,EC San Francisco,37.790314,-122.401915,CityOne Limo Service,37.792820,-122.398191,Punch Line Comedy Club,37.795486,-122.400068,Basketball Court,37.781084,-122.414131,Village Pets & Supplies,37.790433,-122.417048
2,Starbucks,37.794601,-122.402669,The International Dermal Institute,37.790157,-122.402694,Eternity Limousine Service,37.793855,-122.397033,Millennium Tower Club Level,37.790744,-122.395846,Soma Rec Center,37.778838,-122.406328,P.L.A.Y. Pet Lifestyle And You,37.785801,-122.397275
3,Starbucks,37.789098,-122.408438,Product School San Francisco,37.796390,-122.402344,InterContinental San Francisco,37.781998,-122.404722,Great American Music Hall,37.784806,-122.418866,Equinox Sports Club San Francisco,37.786373,-122.404335,Jeffrey's Natural Pet Foods,37.801681,-122.411332
4,Starbucks,37.783998,-122.407206,Voice One,37.796688,-122.402290,Hotel Zelos San Francisco,37.784936,-122.405612,Gold Club,37.785979,-122.399652,Joe Dimaggio Playground Clbhs,37.802749,-122.412327,Men's Grooming Zone,37.786941,-122.405874
5,Starbucks,37.789293,-122.406729,John Yehall Chin Elementary School,37.798585,-122.403119,The Westin St Francis San Francisco on Union S...,37.787738,-122.408419,Club Six,37.781498,-122.409567,Mission Creek Park,37.770963,-122.398624,Wags Pet Wash & Boutique,37.793688,-122.421040
6,Starbucks,37.789534,-122.401869,INSEAD - San Francisco Hub,37.778652,-122.393772,Hyatt Regency San Francisco,37.794251,-122.395694,Black Cat,37.783832,-122.414508,21st Amendment Brewery & Restaurant,37.782468,-122.392563,"Doggiedocs.com - Health, Nutrition, Training T...",37.788808,-122.401109
7,Starbucks,37.787664,-122.401113,First Class Cooking,37.777236,-122.395624,Schroeder's,37.793931,-122.398614,Cat Club,37.775329,-122.409920,Bay Club San Francisco,37.803376,-122.402828,Pet Lifestyle and You,37.789307,-122.399704
8,Starbucks,37.786145,-122.405612,Jean Parker Elementary School,37.797687,-122.411024,Hotel Nikko San Francisco,37.785964,-122.409257,WW Studio,37.801000,-122.409141,Fifth Arrow,37.787569,-122.409639,Woodlands Pet Shop,37.788702,-122.392212
9,Starbucks,37.786805,-122.407228,University of California Hastings College of t...,37.780845,-122.415550,Parc 55 Garage (Propark),37.784908,-122.408980,F8 1192 Folsom,37.775284,-122.409994,Victoria Manalo Draves Park,37.777639,-122.407046,Big Heart Pet Brands,37.795383,-122.399274


In [244]:
lat = 37.788668
lon = -122.400558
sf_map = folium.Map(location = [lat, lon], zoom_start = 15)

In [245]:
add_marker(sf_map,lat,lon,"hand-paper","red","white")

#Ideal office
lat1 = 37.7763337
lon1 = -122.3952334
add_marker(sf_map,lat1,lon1,"train","black","white")

for index, row in sf_companies.iterrows():
    if row["Starbucks_n"] == "Starbucks":
        add_marker(sf_map, row["Starbucks_lat"], row["Starbucks_lon"], "coffee", "white", "green")

    if row["Schools_n"]:
        add_marker(sf_map, row["Schools_lat"], row["Schools_lon"], "graduation-cap", "blue", "white")

    if row["Airport_n"]:
        add_marker(sf_map, row["Airport_lat"], row["Airport_lon"], "plane", "gray", "white")

    if row["Clubs_n"]:
        add_marker(sf_map, row["Clubs_lat"], row["Clubs_lon"], "beer", "lightred", "white")

    if row["Basketball_n"]:
        add_marker(sf_map, row["Basketball_lat"], row["Basketball_lon"], "fa-basketball-ball'", "orange", "black")

    if row["Pet_n"]:
        add_marker(sf_map, row["Pet_lat"], row["Pet_lon"], "paw", "purple", "white")

lat = 37.788668
lon = -122.400558
folium.Circle([lat, lon],
                        radius=2000,
                        fill_color='blue'
                    ).add_to(sf_map)
sf_map

In [246]:
sf_map.save("maps/sf_map.html")

### 3rd city. Los Angeles, CA

In [247]:
lat = 34.085749
lon = -118.343791
radius = 3000

def apply_request_starbucks():
    la_starbucks = []
    for i in query: 
        if i == "starbucks":
            result = requests_for_foursquare("starbucks",lat,lon,radius)
            la_starbucks.append(result)
    return la_starbucks

def apply_request_school():
    la_schools = []
    for i in query:
        if i == "schools":
            result = requests_for_foursquare("schools",lat,lon,radius)
            la_schools.append(result)
    return la_schools

def apply_request_airports():
    la_airport = []
    for i in query:
        if i == "airport":
            result = requests_for_foursquare("airport",lat,lon,radius)
            la_airport.append(result)
    return la_airport

def apply_request_clubs():
    la_clubs = []
    for i in query:
        if i == "club":
            result = requests_for_foursquare("club",lat,lon,radius)
            la_clubs.append(result)
    return la_clubs

def apply_request_vegan():
    la_vegan = []
    for i in query:
        if i == "vegan":
            result = requests_for_foursquare("vegan",lat,lon,radius)
            la_vegan.append(result)
    return la_vegan

def apply_request_basketball():
    la_basketball = []
    for i in query:
        if i == "basketball":
            result = requests_for_foursquare("basketball",lat,lon,radius)
            la_basketball.append(result)
    return la_basketball

def apply_request_pet():
    la_pet = []
    result = requests_for_foursquare_2queries("pet","grooming",lat,lon,radius)
    la_pet.append(result)  
    return la_pet

In [248]:
la_dict = {}

la_starbucks = apply_request_starbucks()
starbucks_names = [result["name"] for i in la_starbucks for result in i.get("results", [])]
la_dict["Starbucks_n"] = starbucks_names
starbucks_latitudes = [result["geocodes"]["main"]["latitude"] for i in la_starbucks for result in i.get("results", [])]
la_dict["Starbucks_lat"] = starbucks_latitudes
starbucks_longitudes = [result["geocodes"]["main"]["longitude"] for i in la_starbucks for result in i.get("results", [])]
la_dict["Starbucks_lon"] = starbucks_longitudes

la_schools = apply_request_school()
schools_names = [result["name"] for i in la_schools for result in i.get("results", [])]
la_dict["Schools_n"] = schools_names
schools_latitudes = [result["geocodes"]["main"]["latitude"] for i in la_schools for result in i.get("results", [])]
la_dict["Schools_lat"] = schools_latitudes
schools_longitudes = [result["geocodes"]["main"]["longitude"] for i in la_schools for result in i.get("results", [])]
la_dict["Schools_lon"] = schools_longitudes

la_airport = apply_request_airports()
airport_name = [result["name"] for i in la_airport for result in i.get("results", [])]
la_dict["Airport_n"] = airport_name
airport_latitude = [result["geocodes"]["main"]["latitude"] for i in la_airport for result in i.get("results", [])]
la_dict["Airport_lat"] = airport_latitude
airport_longitude = [result["geocodes"]["main"]["longitude"] for i in la_airport for result in i.get("results", [])]
la_dict["Airport_lon"] = airport_longitude

la_clubs = apply_request_clubs()
clubs_name = [result["name"] for i in la_clubs for result in i.get("results", [])]
la_dict["Clubs_n"] = clubs_name
club_latitude = [result["geocodes"]["main"]["latitude"] for i in la_clubs for result in i.get("results", [])]
la_dict["Clubs_lat"] = club_latitude
club_longitude = [result["geocodes"]["main"]["longitude"] for i in la_clubs for result in i.get("results",[])]
la_dict["Clubs_lon"] = club_longitude

la_basketball = apply_request_basketball()
basketball_name = [result["name"] for i in la_basketball for result in i.get("results",[])]
la_dict["Basketball_n"] = basketball_name
basketball_latitude = [result["geocodes"]["main"]["latitude"] for i in la_basketball for result in i.get("results", [])]
la_dict["Basketball_lat"] = basketball_latitude
basketball_longitde = [result["geocodes"]["main"]["longitude"] for i in la_basketball for result in i.get("results", [])]
la_dict["Basketball_lon"] = basketball_longitde

la_pet = apply_request_pet()
pet_name = [result["name"] for i in la_pet for result in i.get("results",[])]
la_dict["Pet_n"] = pet_name
pet_latitude = [result["geocodes"]["main"]["latitude"] for i in la_pet for result in i.get("results", [])]
la_dict["Pet_lat"] = pet_latitude
pet_longitde = [result["geocodes"]["main"]["longitude"] for i in la_pet for result in i.get("results", [])]
la_dict["Pet_lon"] = pet_longitde

la_companies = pd.DataFrame(la_dict)
la_companies

,Starbucks_n,Starbucks_lat,Starbucks_lon,Schools_n,Schools_lat,Schools_lon,Airport_n,Airport_lat,Airport_lon,Clubs_n,Clubs_lat,Clubs_lon,Basketball_n,Basketball_lat,Basketball_lon,Pet_n,Pet_lat,Pet_lon
0,Starbucks,34.075883,-118.344968,VS3 - VSS Studios,34.087868,-118.344493,California Tour Lines,34.102303,-118.329913,Academy Night Club,34.102009,-118.320849,SMBCCS Basketball Courts,34.090380,-118.314966,The Urban Pet,34.076414,-118.353962
1,Starbucks,34.098146,-118.343489,Bancroft Middle School,34.088039,-118.336643,In-N-Out Burger,34.098241,-118.341722,The Groundlings Theatre & School,34.083755,-118.349012,Weingart Youth Center,34.102104,-118.319082,Kings & Queens Unisex Grooming Bar,34.083725,-118.346959
2,Starbucks,34.083539,-118.356152,BESTtrafficschool.com,34.097772,-118.346453,Los Angeles County Museum of Art,34.063656,-118.359280,Hudson Theatres,34.090931,-118.332241,Salvation Army,34.102058,-118.319435,Hammer & Nails Grooming Shop for Guys,34.090812,-118.345319
3,Starbucks,34.090394,-118.326288,Hollywood High School,34.099106,-118.340182,FlyAway - Hollywood to LAX,34.099939,-118.324985,The Highlight Room,34.100064,-118.330153,OMG Sports Los Angeles,34.087743,-118.350008,Hammer & Nails - Hand & Foot Grooming Shop,34.083890,-118.369316
4,Starbucks,34.098717,-118.348452,Fairfax High School,34.082054,-118.359943,007 Lax Airport Shuttle,34.090430,-118.341783,Sound Nightclub,34.101016,-118.335832,Plummer Park,34.092911,-118.351099,The Urban Pet,34.091353,-118.343428
5,Starbucks,34.070048,-118.343582,Walt Whitman High School,34.080083,-118.359268,LAX Airport Rides,34.096340,-118.344869,Hollywood Improv,34.083549,-118.367471,Pan Pacific Park,34.076293,-118.355516,D Pet Hotels,34.090055,-118.338946
6,Starbucks,34.101741,-118.337331,Episcopal School of Los Angeles,34.091025,-118.327573,Santa Monica Airport Antique & Collectibles Ma...,34.075066,-118.342194,Def Noodles Comedy Club,34.097758,-118.352679,Barney's Beanery,34.090837,-118.374616,Laurel Pet Hospital,34.090601,-118.364000
7,Starbucks,34.061893,-118.338898,Blessed Sacrament School,34.098155,-118.334503,Airport ASAP Limousine Service,34.097028,-118.354221,Laugh Factory,34.098257,-118.364504,Buffalo Wild Wings,34.101362,-118.343442,Cahuenga Pet Hospital,34.086137,-118.329107
8,Starbucks,34.097423,-118.322460,Samuel A Fryer Yavneh Hebrew Academy,34.070303,-118.336231,FUJETblue.com & Associates,34.098217,-118.345330,Hollywood Palladium,34.098477,-118.324249,Berkshire House,34.074790,-118.344311,Furtographs Pet Portraiture,34.076414,-118.353962
9,Starbucks,34.074908,-118.323373,Los Angeles Make-Up School (LAMUS),34.100580,-118.330870,Airport Shuttle Service,34.091069,-118.364151,Jazz at LACMA,34.063424,-118.359015,Marlborough School - Combs Gymnasium,34.070055,-118.326818,L. A. Pet Clinic,34.083640,-118.327998


In [249]:
lat = 34.085749
lon =  -118.343791
la_map = folium.Map(location = [lat, lon], zoom_start = 15)

In [255]:
add_marker(la_map,lat,lon,"hand-paper","red","white")

#Ideal office
lat1 = 34.062443
lon1 = -118.2938157
add_marker(sf_map,lat1,lon1,"train","black","white")

for index, row in la_companies.iterrows():
    if row["Starbucks_n"] == "Starbucks":
        add_marker(la_map, row["Starbucks_lat"], row["Starbucks_lon"], "coffee", "white", "green")

    if row["Schools_n"]:
        add_marker(la_map, row["Schools_lat"], row["Schools_lon"], "graduation-cap", "blue", "white")

    if row["Airport_n"]:
        add_marker(la_map, row["Airport_lat"], row["Airport_lon"], "plane", "gray", "white")

    if row["Clubs_n"]:
        add_marker(la_map, row["Clubs_lat"], row["Clubs_lon"], "beer", "lightred", "white")

    if row["Basketball_n"]:
        add_marker(la_map, row["Basketball_lat"], row["Basketball_lon"], "fa-basketball-ball'", "orange", "black")

    if row["Pet_n"]:
        add_marker(la_map, row["Pet_lat"], row["Pet_lon"], "paw", "purple", "white")


lat = 34.085749
lon =  -118.343791
folium.Circle([lat, lon],
                        radius=3000,
                        fill_color='blue'
                    ).add_to(la_map)   

la_map

In [254]:
la_map.save("maps/la_map.html")